In [4]:
import cv2
import numpy as np
import Car
import time

# Input and output counters
cnt_up   = 0
cnt_down = 0

cap = cv2.VideoCapture("output.avi")

for i in range(19):
    print (i, cap.get(i))
cv2.ocl.setUseOpenCL(False)

h = cap.get(4)
w = cap.get(3)
frameArea = h*w
areaTH = 350
print ('Area Threshold', areaTH)

#In / out lines
line_up = int(6*(h/20))
line_down   = int(10*(h/20))

up_limit =   int(3*(h/20))
down_limit = int(14*(h/20))

print ("Red line y:",str(line_down))
print ("Blue line y:", str(line_up))
line_down_color = (255,0,0)
line_up_color = (0,0,255)
pt1 =  [0, line_down];
pt2 =  [w, int(12*(h/20))];
pts_L1 = np.array([pt1,pt2], np.int32)
pts_L1 = pts_L1.reshape((-1,1,2))
pt3 =  [0, line_up];
pt4 =  [w, int(7*(h/20))];
pts_L2 = np.array([pt3,pt4], np.int32)
pts_L2 = pts_L2.reshape((-1,1,2))

pt5 =  [0, up_limit];
pt6 =  [w, int(4*(h/20))];
pts_L3 = np.array([pt5,pt6], np.int32)
pts_L3 = pts_L3.reshape((-1,1,2))
pt7 =  [0, down_limit];
pt8 =  [w, int(16*(h/20))];
pts_L4 = np.array([pt7,pt8], np.int32)
pts_L4 = pts_L4.reshape((-1,1,2))

#Variables
font = cv2.FONT_HERSHEY_SIMPLEX
cars = []
max_p_age = 5
pid = 1

subtractor = cv2.createBackgroundSubtractorMOG2(history=136 , varThreshold=50 , detectShadows=False)

while (cap.isOpened):
    ret, frame = cap.read()
    
    for i in cars:
        i.age_one() #age every object one frame
    try:
        kernel = np.ones((7, 7), np.uint8) 
        mask = subtractor.apply(frame)
        closing = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    except:
        print('EOF')
        print ('UP:',cnt_up)
        print ('DOWN:',cnt_down)
        break

    (contours, hierarchy) = cv2.findContours(closing.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > areaTH:
            #################
            #   TRACKING    #
            #################
            
            M = cv2.moments(cnt)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            x,y,w,h = cv2.boundingRect(cnt)

            new = True
            if cy in range(up_limit,down_limit):
                for i in cars:
                    if abs(cx-i.getX()) <= w and abs(cy-i.getY()) <= h:
                        # the object is close to one that was already detected before
                        new = False
                        i.updateCoords(cx,cy)   #Update coordinates on the object and resets age
                        if i.going_UP(line_down,line_up) == True:
                            cnt_up += 1;
                            print ("ID:",i.getId(),'crossed going up at',time.strftime("%c"))
                        elif i.going_DOWN(line_down,line_up) == True:
                            cnt_down += 1;
                            print ("ID:",i.getId(),'crossed going down at',time.strftime("%c"))
                        break
                    if i.getState() == '1':
                        if i.getDir() == 'down' and i.getY() > down_limit:
                            i.setDone()
                        elif i.getDir() == 'up' and i.getY() < up_limit:
                            i.setDone()
                    if i.timedOut():
                        index = cars.index(i)
                        cars.pop(index)
                        del i     
                if new == True:
                    p = Car.MyCar(pid,cx,cy, max_p_age)
                    cars.append(p)
                    pid += 1     
            cv2.circle(frame,(cx,cy), 5, (0,0,255), -1)
            img = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)            
    for i in cars:
        cv2.putText(frame, str(i.getId()),(i.getX(),i.getY()),font,0.3,i.getRGB(),1,cv2.LINE_AA)
        
    #################
    #   IMAGES      #
    #################
    str_up = 'UP: '+ str(cnt_up)
    str_down = 'DOWN: '+ str(cnt_down)
    frame = cv2.polylines(frame,[pts_L1],False,line_down_color,thickness=2)
    frame = cv2.polylines(frame,[pts_L2],False,line_up_color,thickness=2)
    frame = cv2.polylines(frame,[pts_L3],False,(255,255,255),thickness=1)
    frame = cv2.polylines(frame,[pts_L4],False,(255,255,255),thickness=1)
    cv2.putText(frame, str_up ,(10,40),font,0.5,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(frame, str_up ,(10,40),font,0.5,(0,0,255),1,cv2.LINE_AA)
    cv2.putText(frame, str_down ,(10,90),font,0.5,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(frame, str_down ,(10,90),font,0.5,(255,0,0),1,cv2.LINE_AA)
    
    cv2.imshow("mask", mask)
    cv2.imshow("Closing", closing)
    cv2.imshow("Frame", frame)

    key = cv2.waitKey(30)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

0 0.0
1 0.0
2 0.06666666666666667
3 480.0
4 360.0
5 15.0
6 1196444237.0
7 2048.0
8 0.0
9 0.0
10 0.0
11 0.0
12 0.0
13 0.0
14 0.0
15 0.0
16 0.0
17 0.0
18 0.0
Area Threshold 350
Red line y: 180
Blue line y: 108
ID: 1 crossed going up at Tue May 28 00:34:22 2019
ID: 5 crossed going up at Tue May 28 00:34:23 2019
ID: 7 crossed going down at Tue May 28 00:34:23 2019
ID: 8 crossed going down at Tue May 28 00:34:23 2019
ID: 7 crossed going down at Tue May 28 00:34:24 2019
ID: 10 crossed going down at Tue May 28 00:34:24 2019
ID: 12 crossed going up at Tue May 28 00:34:25 2019
ID: 14 crossed going down at Tue May 28 00:34:26 2019
ID: 18 crossed going up at Tue May 28 00:34:28 2019
ID: 17 crossed going up at Tue May 28 00:34:28 2019
ID: 17 crossed going down at Tue May 28 00:34:29 2019
ID: 23 crossed going down at Tue May 28 00:34:30 2019
ID: 25 crossed going down at Tue May 28 00:34:31 2019
ID: 26 crossed going down at Tue May 28 00:34:31 2019
ID: 29 crossed going down at Tue May 28 00:34:32 20

ID: 253 crossed going down at Tue May 28 00:35:11 2019
ID: 257 crossed going up at Tue May 28 00:35:11 2019
ID: 253 crossed going down at Tue May 28 00:35:11 2019
ID: 260 crossed going down at Tue May 28 00:35:11 2019
ID: 262 crossed going down at Tue May 28 00:35:12 2019
ID: 264 crossed going down at Tue May 28 00:35:12 2019
ID: 267 crossed going down at Tue May 28 00:35:13 2019
ID: 266 crossed going up at Tue May 28 00:35:13 2019
ID: 268 crossed going down at Tue May 28 00:35:14 2019
ID: 272 crossed going up at Tue May 28 00:35:14 2019
ID: 270 crossed going down at Tue May 28 00:35:14 2019
ID: 274 crossed going down at Tue May 28 00:35:14 2019
ID: 274 crossed going up at Tue May 28 00:35:15 2019
ID: 280 crossed going up at Tue May 28 00:35:15 2019
ID: 274 crossed going down at Tue May 28 00:35:15 2019
ID: 279 crossed going down at Tue May 28 00:35:15 2019
ID: 283 crossed going up at Tue May 28 00:35:15 2019
ID: 281 crossed going down at Tue May 28 00:35:16 2019
ID: 287 crossed going 